# Preprocesado



#### Introducción

Este código lee archivos EDF que contienen registros de señales fisiológicas. Itera sobre los registros del paciente S001R01 al S001R14. Para cada archivo, lee la información básica como el número de canales, etiquetas de canal, frecuencia de muestreo y las señales registradas. Luego calcula y muestra la duración total de cada registro en segundos.

In [1]:
import pyedflib
from tabulate import tabulate
import numpy as np
import json
import os

In [2]:
SX = 50 #109
RX = 14

Probamos a registrar los tiempos de los registros del primer paciente

In [3]:
# Leer todos los registros S001R01 a S001R14
for registro in range(1, RX+1):
    file_name = f"../Data/Raw/S001R{registro:02d}.edf"
    try:
        with pyedflib.EdfReader(file_name) as f:
            n_channels = f.signals_in_file
            channel_labels = f.getSignalLabels()
            sample_frequency = f.getSampleFrequency(0)
            signals = [f.readSignal(i) for i in range(n_channels)]
            # Calcular el número total de muestras
            total_samples = len(signals[0])
        
            # Calcular la duración total en segundos
            total_time = total_samples / sample_frequency
            print(f"Tiempo total del archivo {file_name}: {total_time} segundos")
    except Exception as e:
        print(f"Error al leer el archivo {file_name}: {e}")


Tiempo total del archivo ../Data/Raw/S001R01.edf: 61.0 segundos
Tiempo total del archivo ../Data/Raw/S001R02.edf: 61.0 segundos
Tiempo total del archivo ../Data/Raw/S001R03.edf: 125.0 segundos
Tiempo total del archivo ../Data/Raw/S001R04.edf: 125.0 segundos
Tiempo total del archivo ../Data/Raw/S001R05.edf: 125.0 segundos
Tiempo total del archivo ../Data/Raw/S001R06.edf: 125.0 segundos
Tiempo total del archivo ../Data/Raw/S001R07.edf: 125.0 segundos
Tiempo total del archivo ../Data/Raw/S001R08.edf: 125.0 segundos
Tiempo total del archivo ../Data/Raw/S001R09.edf: 125.0 segundos
Tiempo total del archivo ../Data/Raw/S001R10.edf: 125.0 segundos
Tiempo total del archivo ../Data/Raw/S001R11.edf: 125.0 segundos
Tiempo total del archivo ../Data/Raw/S001R12.edf: 125.0 segundos
Tiempo total del archivo ../Data/Raw/S001R13.edf: 125.0 segundos
Tiempo total del archivo ../Data/Raw/S001R14.edf: 125.0 segundos


Registramos los tiempos de 20 pacientes de muestra

In [4]:
# Lista para almacenar los tiempos de los registros de cada paciente
paciente_times = []

# Leer todos los registros S001 a S020
for paciente in range(1, SX+1):
    time = []
    for registro in range(1, RX+1):
        file_name = f"../Data/Raw/S{paciente:03d}R{registro:02d}.edf"
        try:
            with pyedflib.EdfReader(file_name) as f:
                sample_frequency = f.getSampleFrequency(0)
                total_samples = f.getNSamples()[0]
                total_time = total_samples / sample_frequency
                time.append(total_time)
        except Exception as e:
            print(f"Error al leer el archivo {file_name}: {e}")
    
    # Agregar los tiempos al paciente_times
    paciente_times.append([f"S{paciente:03d}", *time])

print("Tiempo de los registros de los pacientes en segundos:\n")
# Imprimir la tabla
headers = ["Paciente", *[f"R{registro:02d}" for registro in range(1, 15)]]
print(tabulate(paciente_times, headers=headers, floatfmt=".2f"))


Tiempo de los registros de los pacientes en segundos:

Paciente      R01    R02     R03     R04     R05     R06     R07     R08     R09     R10     R11     R12     R13     R14
----------  -----  -----  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------
S001        61.00  61.00  125.00  125.00  125.00  125.00  125.00  125.00  125.00  125.00  125.00  125.00  125.00  125.00
S002        61.00  61.00  123.00  123.00  123.00  123.00  123.00  123.00  123.00  123.00  123.00  123.00  123.00  123.00
S003        61.00  61.00  125.00  125.00  125.00  125.00  125.00  125.00  125.00  125.00  125.00  125.00  125.00  125.00
S004        61.00  61.00  123.00  123.00  123.00  123.00  123.00  123.00  123.00  123.00  123.00  123.00  123.00  123.00
S005        61.00  61.00  123.00  123.00  123.00  123.00  123.00  123.00  123.00  123.00  123.00  123.00  123.00  123.00
S006        61.00  61.00  123.00  123.00  123.00  123.00  123.00  123.00  123.00  123.00  123.00  

Guardamos en la variable data los ficheros preprocesados en una lista en 3 niveles

In [ ]:
data = []

for paciente in range(1, SX+1):
    for registro in range(1, RX+1):
        file_name = f"../Data/Raw/S{paciente:03d}R{registro:02d}.edf"
        try:
            
            with pyedflib.EdfReader(file_name) as f:
                n_channels = f.signals_in_file
                channel_labels = f.getSignalLabels()
                sample_frequency = f.getSampleFrequency(0)
                signals = [f.readSignal(i) for i in range(n_channels)]
                
                # Obtener información básica
                n_channels = f.signals_in_file
                channel_labels = f.getSignalLabels()
                # Se asume que todas las señales tienen la misma frecuencia de muestreo
                sample_frequency = f.getSampleFrequency(0)

                # Leer todas las señales
                #signals = np.zeros((n_channels, f.getNSamples()[0]))
                signals = []
                for i in range(n_channels):
                    signal = f.readSignal(i)
                    signals.append(signal.tolist())
                    #signals[i] = f.readSignal(i)
                
                data.append(signals)
                
        except Exception as e:
            print(f"Error al leer el archivo {file_name}: {e}")
            
    print(f"Ficheros del paciente S{paciente:03d}: ¡Guardados correctamente!")
print(f"\nTotal de registros guardados: {len(data)}")

Ficheros del paciente S001: ¡Guardados correctamente!
Ficheros del paciente S002: ¡Guardados correctamente!
Ficheros del paciente S003: ¡Guardados correctamente!
Ficheros del paciente S004: ¡Guardados correctamente!
Ficheros del paciente S005: ¡Guardados correctamente!
Ficheros del paciente S006: ¡Guardados correctamente!
Ficheros del paciente S007: ¡Guardados correctamente!
Ficheros del paciente S008: ¡Guardados correctamente!
Ficheros del paciente S009: ¡Guardados correctamente!
Ficheros del paciente S010: ¡Guardados correctamente!
Ficheros del paciente S011: ¡Guardados correctamente!
Ficheros del paciente S012: ¡Guardados correctamente!
Ficheros del paciente S013: ¡Guardados correctamente!
Ficheros del paciente S014: ¡Guardados correctamente!
Ficheros del paciente S015: ¡Guardados correctamente!
Ficheros del paciente S016: ¡Guardados correctamente!
Ficheros del paciente S017: ¡Guardados correctamente!


Este código imprime el contenido de data de manera estructurada, mostrando los datos de cada registro, canal y señal. 

In [ ]:
print(f"-Datos de registro {len(data)}:")
print(f"-Datos de canales {len(data[0])}:")
print(f"-Datos de señales por canal {len(data[0][0])}:")

Explicación de los niveles de la lista data

In [ ]:
for i, paciente_data in enumerate(data[:2], start=1):
    print(f"-Datos del registro {i}:")
    for j, registro_data in enumerate(paciente_data[:2], start=1):
        print(f"\t-Canal {j}:")
        for k, signal_data in enumerate(registro_data[:2], start=1):
            print(f"\t\t-Señal {k}: {signal_data}")
        print(f"\t\t...")

In [ ]:
# Guardar datos en un archivo JSON
#try: 
#    with open('datos.json', 'w') as archivo:
#        json.dump(data, archivo)
#    print("Datos guardados exitosamente en 'datos.json'.")
#except Exception as e:
#    print(f"Error al guardar los datpd: {e}")

### Padding a la derecha 
En el contexto del aprendizaje automático, el padding es comúnmente utilizado cuando se trabaja con secuencias de longitud variable, como en el procesamiento de texto o en el análisis de series temporales.

Padding es una técnica usada en el procesamiento de secuencias de longitud variable, una situación frecuente en el análisis de datos biomédicos como el EEG. Aplicamos padding a los datos EEG, para asegurarnos de que todas las secuencias tengan la misma longitud.

#### Padding a la derecha

Agregamos ceros a los registros para completar la longitud maxima de las grabaciones.

Ejemplo:
    Registros originales: "123"   y "12345654566"
    Registro con padding a la derecha: "1230000000"
    
#### Padding a la izquierda

Eliminamos los registros finales que superen a la longitud del minimo registro..

Ejemplo:
    Registros originales: "123"   y "12345654566"
    Registro con padding a la derecha: "123"



In [ ]:
# Encontrar el máximo número de valores en un dato
max_len = max(len(registro) for sublist in data for registro in sublist)
max_len

In [ ]:
# Encontrar el mínimo número de valores en un dato
min_len = min(len(registro) for sublist in data for registro in sublist)
min_len

In [ ]:
def padding_right(data):
    max_len = max(len(registro) for sublist in data for registro in sublist)
    data_padding_right = []
    
    # Rellenar los datos faltantes con 0
    for sublist in data:
        padded_sublist = [registro + [0] * (max_len - len(registro)) for registro in sublist]
        data_padding_right.append(padded_sublist)
    
    return data_padding_right

def padding_left(data):
    min_len = min(len(registro) for sublist in data for registro in sublist)
    data_padding_left = []

    # Eliminar registros sobrantes desde el lado derecho
    for sublist in data:
        padded_sublist = [registro[-min_len:] for registro in sublist]
        data_padding_left.append(padded_sublist)
    
    return data_padding_left


In [ ]:
data_padding = padding_left(data)

In [ ]:
len(data_padding[0][0])

data_padding_right = data
#Rellenar los datos faltantes con 0
for sublist in data_padding_right:
    for e in sublist:
        while len(e) < max_len:
            e.append(0)

## Guardar los datos

Por último almacenamos los datos en formato json

In [ ]:
#try:
#    for index, item in enumerate(data):
#        file_name = f'../Data/Preprocessed/S{index + 1:03}.json'
#        with open(file_name, 'w') as archivo:
#            json.dump(item, archivo)
#        print(f"Datos cargados exitosamente desde '{ruta_archivo}'.")
#    print(f"Datos del índice {index} guardados exitosamente en '{file_name}'.")
#except Exception as e:
#    print(f"Error al guardar los datos: {e}")

In [ ]:
try:
    patientx = 1
    registerx = 1
    for index, item in enumerate(mata_padding):
        
        # No guardamos los dos registros iniciales
        if(registerx>2):
            file_name = f'../Data/Preprocessed/S{patientx:03}R{registerx:03}_raw.json'
            with open(file_name, 'w') as archivo:
                json.dump(item, archivo)
        
            print(f"Datos cargados exitosamente en 'S{patientx:03}R{registerx:03}_raw.json'.")
        
        if(registerx<14):
            registerx+=1
        else:
            patientx += 1
            registerx = 1
        
    print(f"Todos los datos han sido guardados correctamente.")
    
except Exception as e:
    print(f"Error al guardar los datos: {e}")
